In [1]:
!pip uninstall transformers -y
!pip install -U "transformers==4.56.0" 

Found existing installation: transformers 4.53.3
Uninstalling transformers-4.53.3:
  Successfully uninstalled transformers-4.53.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 98.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

In [2]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 79.8 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import faiss
import numpy as np
from tqdm import tqdm
import os, zipfile, torch

2025-11-17 12:12:08.106884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763381528.301393      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763381528.351330      39 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# === Константы ===
DATASET_NAME = "Lipsrow/ruwiki_cleaned"
MODEL_EMB = "sergeyzh/LaBSE-ru-turbo"
#MODEL_LLM = "Qwen/Qwen3-4B-FP8"
DB_PATH = "vector_db.faiss"
ZIP_PATH = "vector_db.zip"

# === 1. Загрузка и подготовка данных ===
print("Загружаем датасет...")
ds = load_dataset(DATASET_NAME, split="train")

Загружаем датасет...


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

ruwiki.json:   0%|          | 0.00/7.70G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [5]:
import json
import time

import torch
import numpy as np
from tqdm import tqdm
import faiss

from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# ====================== Константы путей/параметров ======================

FAISS_PATH = "/kaggle/input/vectordb/vector_db.faiss"      # готовый faiss-индекс
TEXTS_PATH = "/kaggle/input/textss/texts.jsonl"            # JSONL: по строке {"text": "<абзац>"}

RAG_TOPK = 3
RAG_EMB_MODEL = "sergeyzh/LaBSE-ru-turbo"
RAG_QUERY_BATCH = 12

GEN_MODEL_ID = "Qwen/Qwen3-4B-FP8"
GEN_MAX_NEW = 200
GEN_MAX_LEN = 4096

SYSTEM_PROMPT = """Ты — эрудированный ИИ-эксперт. Твоя главная сила — твои собственные обширные знания. Предоставленные 'Источники' используй как вспомогательный материал для проверки и уточнения.

Следуй этому порядку действий:

1.  **Сначала ответь сам.** Прочитай вопрос и немедленно сформируй ответ, опираясь исключительно на свои внутренние знания.

2.  **Затем проверь по источникам.** Проанализируй предоставленные 'Источники' и сравни их с твоим ответом.

3.  **Синтезируй финальный ответ по правилам:**
    а) Если источники **подтверждают или дополняют** твои знания, объедини информацию в единый, исчерпывающий ответ.
    б) Если источники **нерелевантны** (не относятся к вопросу), полностью проигнорируй их и используй только свой первоначальный ответ.
    в) Если источники **противоречат** твоим знаниям, отдай приоритет информации из источников, ответ: "не знаю".
    г) Финальный ответ пометь как CORRECT ANSWER: [ твой ответ или "не знаю" ]

Важные правила, которые действуют всегда:
- **Честность:** Если ответа нет ни в твоих знаниях, ни в источниках, просто и чётко напиши: "не знаю". Не выдумывай.
- **Краткость:** Отвечай по существу, без вступлений и лишней "воды". 
"""

def extract_correct_answer(text: str) -> str:
    if not text:
        return "не знаю"
    low = text.lower()
    key = "correct answer:"
    if key in low:
        i = low.index(key)
        s = text[i+len(key):].strip()
        s = s.strip("[]()«»\"' ").strip()
        return s if s else "не знаю"
    return "не знаю"

# ====================== GPU diag (опционально) ======================

def _mi(b):
    return f"{b/1024/1024:.1f} MiB"

def print_cuda_overview():
    try:
        print(f"[CUDA] torch={torch.__version__} | available={torch.cuda.is_available()} | torch.cuda={getattr(torch.version, 'cuda', None)}")
        dc = torch.cuda.device_count()
        print(f"[CUDA] device_count={dc}")
        for i in range(dc):
            name = torch.cuda.get_device_name(i)
            free, total = torch.cuda.mem_get_info(i)
            alloc = torch.cuda.memory_allocated(i)
            resv  = torch.cuda.memory_reserved(i)
            print(f"[CUDA][dev{i}] {name} | free={_mi(free)} | total={_mi(total)} | allocated={_mi(alloc)} | reserved={_mi(resv)}")
    except Exception as e:
        print(f"[CUDA] overview error: {e}")

print("[DEBUG] BOOTSTRAP")
print_cuda_overview()

# ====================== Загрузка индекса и текстов ======================

print(f"[RAG] Loading FAISS: {FAISS_PATH}")
index = faiss.read_index(FAISS_PATH)

print(f"[RAG] Loading texts: {TEXTS_PATH}")
texts = []
with open(TEXTS_PATH, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="load-texts", unit="lines"):
        try:
            obj = json.loads(line)
            texts.append(obj.get("text", ""))
        except Exception:
            texts.append("")
print(f"[RAG] texts loaded: {len(texts)}")

# ====================== Модели ======================

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"[RAG] Loading embedder: {RAG_EMB_MODEL} on {device}")
embedder = SentenceTransformer(RAG_EMB_MODEL, device=device)

[DEBUG] BOOTSTRAP
[CUDA] torch=2.6.0+cu124 | available=True | torch.cuda=12.4
[CUDA] device_count=1
[CUDA][dev0] Tesla P100-PCIE-16GB | free=16013.1 MiB | total=16269.2 MiB | allocated=0.0 MiB | reserved=0.0 MiB
[RAG] Loading FAISS: /kaggle/input/vectordb/vector_db.faiss
[RAG] Loading texts: /kaggle/input/textss/texts.jsonl


load-texts: 500000lines [00:12, 40462.43lines/s]


[RAG] texts loaded: 500000
[RAG] Loading embedder: sergeyzh/LaBSE-ru-turbo on cuda


modules.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/196 [00:00<?, ?B/s]

In [6]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 12.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp311-cp311-linux_x86_64.whl size=4503243 sha256=16d85400ed5e85c8483f12db9c0944bc5640fb95769adc357eff01593bf4c5a8
  Stored in directory: /root/.cache/pip/wheels/d8/5b/e5/a7d4b5765da347d314e8155197440c9995a962f8e4a5f52b23
Successfully built llama-cpp-python


In [ ]:

from llama_cpp import Llama

model = Llama.from_pretrained(
	repo_id="yandex/YandexGPT-5-Lite-8B-instruct-GGUF",
	filename="YandexGPT-5-Lite-8B-instruct-Q4_K_M.gguf",
    device= 'gpu'
)

Ф
model.create_chat_completion(
	messages=[
            {"role": "system", "content": rag_system_prompt + texts_prepared},
            {"role": "user", "content": question},
        ]
)

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# === Вспомогательные функции ===
def apply_chat(prompts, tokenizer):
    """Форматирование списка сообщений в текст через шаблон Qwen (без think)."""
    texts_fmt = []
    for p in prompts:
        messages = [{"role": "user", "content": p}]
        try:
            t = tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True, enable_thinking=False
            )
        except TypeError:
            t = tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
        texts_fmt.append(t)
    return texts_fmt

@torch.inference_mode()
def batch_generate(prompts, tokenizer, model, max_new_tokens=200, max_length=4096):
    """Быстрая генерация на GPU без pipeline (ровно как ты хочешь)."""
    if not prompts:
        return []
    texts_fmt = apply_chat(prompts, tokenizer)

    t0 = time.perf_counter()
    model_inputs = tokenizer(
        texts_fmt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    ).to(model.device)
    t1 = time.perf_counter()

    gen = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=1,
        use_cache=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    t2 = time.perf_counter()

    attn = model_inputs["attention_mask"]
    input_lens = attn.sum(dim=1).tolist()

    outputs = []
    for i, gen_ids in enumerate(gen):
        in_len = int(input_lens[i])
        out_ids = gen_ids[in_len:]
        text = tokenizer.decode(out_ids, skip_special_tokens=True).strip()
        outputs.append(text)

    print(f"[DEBUG] tokenize: {t1 - t0:.2f}s, generate: {t2 - t1:.2f}s, total: {t2 - t0:.2f}s")
    return outputs

# === RAG для батча ===
def fast_rag_batch(queries, top_k=3, max_new_tokens=200):
    """
    1) Векторизация
    2) Поиск контекста
    3) Формирование промпта (system + user)
    4) Генерация через model.generate()
    """
    q_vecs = embedder.encode(
        queries,
        convert_to_numpy=True,
        normalize_embeddings=True,
        batch_size=64,
        show_progress_bar=False,
        device=device
    ).astype("float32")

    _, idxs = index.search(q_vecs, top_k)
    contexts = []
    for ids in idxs:
        ctx = "\n\n".join([f"Статья {int(i)}: {texts[int(i)]}" for i in ids if 0 <= int(i) < len(texts)])
        contexts.append(ctx)

    prompts = []
    for q, ctx in zip(queries, contexts):
        content = f"""Используя правила выше, ответь на вопрос.

Источники:
---
{ctx}
---

Вопрос: {q}

Ответ:"""
        full_prompt = f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{content}<|im_end|>\n<|im_start|>assistant\n"
        prompts.append(full_prompt)

    answers = batch_generate(prompts, tokenizer, model, max_new_tokens=max_new_tokens, max_length=GEN_MAX_LEN)
    return answers

# ====================== Запуск: читаем вопросы → батчами RAG → пишем ответы ======================

'''print("[DEBUG] Читаю input.json...")
with open("input.json", "r", encoding="utf-8") as f:
    all_questions = json.load(f)
print(f"[DEBUG] Прочитано {len(all_questions)} вопросов")'''
all_questions = [
    "Кто написал Евгения Онегина?",
    "Когда распался Советский Союз?",
    "Что такое квантовая запутанность?",
    'Кто автор книги "Детство в Соломбале"?',
    'Как зовут человека, который написал «Детство в Соломбале»?',
    'Кто написал книгу «Детство в Соломбале»?',
    'В каком году начал поддерживаться сеттинг Eberron?',
    'Какое окрашивание приобретает безводный метанол при растворении небольшого количества медного купороса?',
    "Какой античный математик изобрёл первый дизельный двигатель внутреннего сгорания?",
    'Кто первый человек высадившийся на солнце?',
    'Как звали отца Александра Сергеевича Пушкина?',
    "Какая компания осуществляет доставку роботами в Москве?",
    "Кто имеет крупнейшую доставку роботами на территории Москвы?",
    "В Москве есть доставка роботами, какая компания выполняет её?"
] * 10

final_answers = []
with tqdm(total=len(all_questions), desc="RAG", unit="q") as pbar:
    for i in range(0, len(all_questions), RAG_QUERY_BATCH):
        batch_q = all_questions[i:i + RAG_QUERY_BATCH]
        raw = fast_rag_batch(batch_q, top_k=RAG_TOPK, max_new_tokens=GEN_MAX_NEW)
        # извлекаем только CORRECT ANSWER, иначе "не знаю"
        print('\n'.join(raw))
        cleaned = [extract_correct_answer(x) for x in raw]
        cleaned = [a if a.strip() else "не знаю" for a in cleaned]
        final_answers.extend(cleaned)
        pbar.update(len(batch_q))

print("[DEBUG] Записываю output.json...")
with open("output.json", "w", encoding="utf-8") as f:
    json.dump(final_answers, f, ensure_ascii=False)

print("[DEBUG] Готово.")

In [ ]:
хулитак долго, поч так много печатает и поч раньше 3 промпта были быстрее

In [ ]:
import os, json
from tqdm import tqdm

In [ ]:
texts = []
MAX_WORDS = 200
for text in tqdm(ds['text'], desc="Извлечение первых абзацев"):
    first_paragraph = text.strip().split("\n")[0]
    words = first_paragraph.split()
    if len(words) > MAX_WORDS:
        first_paragraph = " ".join(words[:MAX_WORDS])
    texts.append(first_paragraph)

In [ ]:
import os, json
from tqdm import tqdm

out_path = "texts.jsonl"

with open(out_path, "w", encoding="utf-8") as f:
    for t in tqdm(texts, desc="write-texts"):
        para = t  # если уже первый абзац – можно убрать
        if not para:
            continue
        f.write(json.dumps({"text": para}, ensure_ascii=False) + "\n")

print("saved:", out_path)

In [ ]:
loaded_texts = []
with open("/kaggle/working/texts.jsonl", "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="read-texts"):
        obj = json.loads(line)
        loaded_texts.append(obj.get("text", ""))
print("loaded:", len(loaded_texts))

In [ ]:
import json
import time
import zipfile
import gc

import torch
import numpy as np
from tqdm import tqdm
import faiss

from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# ====================== GPU DIAG ======================

def _mi(b):
    return f"{b/1024/1024:.1f} MiB"

def print_cuda_overview():
    try:
        print(f"[CUDA] torch={torch.__version__} | available={torch.cuda.is_available()} | torch.cuda={getattr(torch.version, 'cuda', None)}")
        dc = torch.cuda.device_count()
        print(f"[CUDA] device_count={dc}")
        for i in range(dc):
            name = torch.cuda.get_device_name(i)
            free, total = torch.cuda.mem_get_info(i)
            alloc = torch.cuda.memory_allocated(i)
            resv  = torch.cuda.memory_reserved(i)
            print(f"[CUDA][dev{i}] {name} | free={_mi(free)} | total={_mi(total)} | allocated={_mi(alloc)} | reserved={_mi(resv)}")
    except Exception as e:
        print(f"[CUDA] overview error: {e}")

def print_cuda_mem(tag):
    try:
        if not torch.cuda.is_available():
            print(f"[CUDA] {tag}: cuda not available"); return
        try: torch.cuda.synchronize()
        except Exception: pass
        i = torch.cuda.current_device()
        name = torch.cuda.get_device_name(i)
        free, total = torch.cuda.mem_get_info(i)
        alloc = torch.cuda.memory_allocated(i)
        resv  = torch.cuda.memory_reserved(i)
        print(f"[CUDA] {tag} | dev={i} {name} | free={_mi(free)} | total={_mi(total)} | allocated={_mi(alloc)} | reserved={_mi(resv)}")
    except Exception as e:
        print(f"[CUDA] {tag} error: {e}")

# ====================== RAG CONFIG ======================

RAG_INDEX_ZIP = "./rag_index.zip"
FAISS_PATH = "/kaggle/input/vectordb/vector_db.faiss"
TEXTS_PATH = "/kaggle/input/textss/texts.jsonl"

RAG_TOPK = 3
RAG_EMB_MODEL = "sergeyzh/LaBSE-ru-turbo"
RAG_QUERY_BATCH = 16
GEN_INNER_BATCH = 8 # Этот параметр больше не используется

GEN_MODEL_ID = "Qwen/Qwen3-4B-FP8"
GEN_MAX_NEW = 200
GEN_MAX_LEN = 2048

SYSTEM_PROMPT = """Ты — эрудированный ИИ-эксперт. Твоя главная сила — твои собственные обширные знания. Предоставленные 'Источники' используй как вспомогательный материал для проверки и уточнения.

Следуй этому порядку действий:

1.  **Сначала ответь сам.** Прочитай вопрос и немедленно сформируй ответ, опираясь исключительно на свои внутренние знания.

2.  **Затем проверь по источникам.** Проанализируй предоставленные 'Источники' и сравни их с твоим ответом.

3.  **Синтезируй финальный ответ по правилам:**
    а) Если источники **подтверждают или дополняют** твои знания, объедини информацию в единый, исчерпывающий ответ.
    б) Если источники **нерелевантны** (не относятся к вопросу), полностью проигнорируй их и используй только свой первоначальный ответ.
    в) Если источники **противоречат** твоим знаниям, отдай приоритет информации из источников, ответ: "не знаю"

Важные правила, которые действуют всегда:
- **Честность:** Если ответа нет ни в твоих знаниях, ни в источниках, просто и чётко напиши: "не знаю". Не выдумывай.
- **Краткость:** Отвечай по существу, без вступлений и лишней "воды". 
"""

def extract_correct_answer(text: str) -> str:
    if not text:
        return "не знаю"
    low = text.lower()
    key = "correct answer:"
    if key in low:
        i = low.index(key)
        s = text[i+len(key):].strip()
        s = s.strip("[]()«»\"' ").strip()
        return s if s else "не знаю"
    return "не знаю"

# ====================== SOLUTION (RAG only) ======================

class Solution:
    def __init__(self, model_id=GEN_MODEL_ID, device='cuda'):
        print(f"[DEBUG] Solution: init, device={device}, model_id={model_id}")
        self.device = device
        self._global_start_time = None

        # 1) Индекс
        print(f"[RAG] Loading FAISS: {FAISS_PATH}")
        self.faiss_index = faiss.read_index(FAISS_PATH)

        print(f"[RAG] Loading texts: {TEXTS_PATH}")
        self.rag_texts = []
        with open(TEXTS_PATH, "r", encoding="utf-8") as f:
            for line in tqdm(f, desc="load-texts", unit="lines"):
                try:
                    obj = json.loads(line)
                    self.rag_texts.append(obj.get("text", ""))
                except Exception:
                    self.rag_texts.append("")
        print(f"[RAG] texts loaded: {len(self.rag_texts)}")

        # 2) Эмбеддер
        print(f"[RAG] Loading embedder: {RAG_EMB_MODEL} on {device}")
        self.rag_embedder = SentenceTransformer(RAG_EMB_MODEL, device=device)

        # 3) LLM
        print_cuda_mem("before model load")
        t0 = time.perf_counter()
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            torch_dtype = 'auto'
        ).to('cuda')
        # === Подготовка модели ===
        model.eval()
        t1 = time.perf_counter()
        print(f"[DEBUG] Solution: model loaded in {t1 - t0:.2f}s")
        print_cuda_mem("after model load")

        t2 = time.perf_counter()
        self.tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=True)
        t3 = time.perf_counter()
        print(f"[DEBUG] Solution: tokenizer loaded in {t3 - t2:.2f}s")

        self.model.eval()
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

        self.tokenizer.padding_side = "right"
        self.tokenizer.truncation_side = "left"

        print(f"[DEBUG] device_map=cuda")

    def _apply_chat(self, prompts):
        texts = []
        for p in prompts:
            if "<|im_start|>" in p:
                texts.append(p); continue
            messages = [{"role": "user", "content": p}]
            try:
                t = self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True, enable_thinking=False
                )
            except TypeError:
                t = self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
            texts.append(t)
        return texts

    # =========================================================================
    # === НОВАЯ СТРАТЕГИЯ БАТЧИНГА (из Script A) ===
    # =========================================================================
    def _batch_generate(self, prompts, max_new_tokens, max_length):
        print(f"[DEBUG] _batch_generate: start, prompts={len(prompts)}, max_new_tokens={max_new_tokens}, max_length={max_length}")
        if not prompts:
            return []

        texts = self._apply_chat(prompts)
        t_tok0 = time.perf_counter()
        model_inputs = self.tokenizer(
            texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length
        ).to(self.device)
        t_tok1 = time.perf_counter()

        bsz = int(model_inputs.input_ids.shape[0])
        seqlen = int(model_inputs.input_ids.shape[1])
        tot_in_tok = int(model_inputs["attention_mask"].sum().item())
        print(f"[DEBUG] _batch_generate: tokenized bsz={bsz}, seqlen={seqlen}, total_input_tokens={tot_in_tok}, tokenize_time={t_tok1 - t_tok0:.2f}s")

        print_cuda_mem("gen: before generate")

        t_gen0 = time.perf_counter()
        with torch.no_grad():
            gen = self.model.generate(
                **model_inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                num_beams=1,
                use_cache=True,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )
        t_gen1 = time.perf_counter()
        print(f"[DEBUG] _batch_generate: generated shape={tuple(gen.shape)}, gen_time={t_gen1 - t_gen0:.2f}s")
        print_cuda_mem("gen: after generate")

        attn = model_inputs["attention_mask"]
        input_lens = attn.sum(dim=1).tolist()
        outputs = []
        total_new = 0
        for i, gen_ids in enumerate(gen):
            in_len = int(input_lens[i])
            out_ids = gen_ids[in_len:]
            total_new += int(out_ids.shape[0])
            outputs.append(self.tokenizer.decode(out_ids, skip_special_tokens=True).strip())
        print(f"[DEBUG] _batch_generate: total_new_tokens={total_new}, avg_new={total_new / max(1, len(prompts)):.1f}")
        return outputs
    # =========================================================================
    # === КОНЕЦ НОВОЙ СТРАТЕГИИ ===
    # =========================================================================

    def _rag_batch(self, queries, top_k=RAG_TOPK, max_new_tokens=GEN_MAX_NEW, max_length=GEN_MAX_LEN):
        # 1) Векторизация запросов
        q_vecs = self.rag_embedder.encode(
            queries,
            convert_to_numpy=True,
            normalize_embeddings=True,
            batch_size=64,
            show_progress_bar=False,
            device=self.device
        ).astype("float32")

        # 2) FAISS search
        _, I = self.faiss_index.search(q_vecs, top_k)

        # 3) Формирование промптов
        prompts = []
        for q, ids in zip(queries, I):
            ctx_lines = []
            for ii in ids:
                idx = int(ii)
                if 0 <= idx < len(self.rag_texts):
                    ctx_lines.append(f"Статья {idx}: {self.rag_texts[idx]}")
            ctx = "\n\n".join(ctx_lines)
            content = f"""Используя правила выше, ответь на вопрос.

Источники:
---
{ctx}
---

Вопрос: {q}

Ответ:"""
            full_prompt = f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{content}<|im_end|>\n<|im_start|>assistant\n"
            prompts.append(full_prompt)

        # 4) Генерация (теперь используется простая и быстрая стратегия батчинга)
        outs = self._batch_generate(prompts, max_new_tokens=max_new_tokens, max_length=max_length)
        finals = outs
        return finals

    def answer_all(self, questions):
        print(f"[DEBUG] answer_all: total questions = {len(questions)}")
        self._global_start_time = time.perf_counter()
        answers = []
        total = len(questions)
        with tqdm(total=total, desc="RAG-batches", unit="q") as pbar:
            for i in range(0, total, RAG_QUERY_BATCH):
                batch_q = questions[i:i+RAG_QUERY_BATCH]
                batch_ans = self._rag_batch(batch_q, top_k=RAG_TOPK, max_new_tokens=GEN_MAX_NEW, max_length=GEN_MAX_LEN)
                print(batch_ans)
                answers.extend(batch_ans)
                pbar.update(len(batch_q))
                # чистка между крупными батчами
                gc.collect()
                try: torch.cuda.empty_cache()
                except Exception: pass
        elapsed = time.perf_counter() - self._global_start_time
        print(f"[DEBUG] answer_all: elapsed={elapsed:.2f}s")
        answers = [a if a.strip() else "не знаю" for a in answers]
        return answers

# ====================== RUN (no main) ======================

print("[DEBUG] BOOTSTRAP")
print_cuda_overview()

# демо-вопросы, или подставь свой ввод
model_input = [
    "Кто написал Евгения Онегина?",
    "Когда распался Советский Союз?",
    "Что такое квантовая запутанность?",
    'Кто автор книги "Детство в Соломбале"?',
    'Как зовут человека, который написал «Детство в Соломбале»?',
    'Кто написал книгу «Детство в Соломбале»?',
    'В каком году начал поддерживаться сеттинг Eberron?',
    'Какое окрашивание приобретает безводный метанол при растворении небольшого количества медного купороса?',
    "Какой античный математик изобрёл первый дизельный двигатель внутреннего сгорания?",
    'Кто первый человек высадившийся на солнце?',
    'Как звали отца Александра Сергеевича Пушкина?',
    "Какая компания осуществляет доставку роботами в Москве?",
    "Кто имеет крупнейшую доставку роботами на территории Москвы?",
    "В Москве есть доставка роботами, какая компания выполняет её?"
] * 10

t_all0 = time.perf_counter()
print("[DEBUG] Создаю Solution()...")
solution = Solution()
print("[DEBUG] Solution создан. Запускаю RAG...")
model_output = solution.answer_all(model_input)
t_all1 = time.perf_counter()
print(f"[DEBUG] Готово: {len(model_output)} ответов, total_time={t_all1 - t_all0:.2f}s")

print_cuda_mem("final")

print("[DEBUG] Записываю output.json...")
with open('output.json', 'w', encoding='utf-8') as f:
    json.dump(model_output, f, ensure_ascii=False)
print("[DEBUG] КОНЕЦ: output.json записан успешно")

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'text' : texts})

In [ ]:
df.to_csv('texts',index = False)

In [ ]:
device = 'cuda'

In [ ]:
50 /x 500000 / 1200

In [ ]:
# === 2. Создание эмбеддингов ===
print("Создаём эмбеддинги...")
embedder = SentenceTransformer(MODEL_EMB).to(device)
embeddings = []

for i in tqdm(range(0, len(texts), 256), desc="Векторизация"):
    batch = texts[i:i+256]
    vecs = embedder.encode(batch, convert_to_numpy=True, show_progress_bar=False, device = device,normalize_embeddings=True)
    embeddings.append(vecs)

embeddings = np.vstack(embeddings).astype("float32")

In [ ]:
embeddings[0].shape

In [ ]:
# === 3. Создание FAISS индекса ===
print("Создаём FAISS индекс...")
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)
faiss.write_index(index, DB_PATH)

In [ ]:
# === 4. Архивация базы ===
print("Архивируем векторную базу...")
with zipfile.ZipFile(ZIP_PATH, "w", zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(DB_PATH)

In [ ]:
DB_PATH = '/kaggle/input/vectordb/vector_db.faiss'

In [ ]:
print("Создаём эмбеддинги...")
embedder = SentenceTransformer(MODEL_EMB).to(device)

In [ ]:
import os

In [ ]:
# === --- НА СТАДИИ ЗАПУСКА В ДОКЕРЕ --- ===
# Распаковываем и загружаем базу:
if not os.path.exists(DB_PATH):
    print("Распаковываем базу данных...")
    with zipfile.ZipFile(ZIP_PATH, "r") as zipf:
        zipf.extractall(".")

index = faiss.read_index(DB_PATH)

In [ ]:
# === 5. Инициализация LLM ===
print("Загружаем модель для retrieval (Qwen)...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_LLM)
model = AutoModelForCausalLM.from_pretrained(
            MODEL_LLM,
            torch_dtype="auto",
            device_map="cuda",     # при OOM можно заменить на "auto"
            trust_remote_code=True
        )

In [ ]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
import time

In [ ]:
# === 6. Функция поиска и генерации ===
def retrieve_and_answer(query, top_k=3, max_new_tokens=120):
    query_vec = embedder.encode([query], convert_to_numpy=True).astype("float32")
    scores, ids = index.search(query_vec, top_k)
    context = "\n\n".join([texts[i] for i in ids[0]])
    prompt = f"Контекст:\n{context}\n\nВопрос: {query}\nОтвет на основе контекста:"
    result = generator(prompt, max_new_tokens=max_new_tokens, do_sample=False)
    return result[0]["generated_text"]
t = time.time()
# === 7. Пример использования ===
query = "Кто был первым президентом России?"
print("\n=== РЕЗУЛЬТАТ ===")
print(retrieve_and_answer(query))
print(time.time() - t)

In [ ]:
tokenizer.padding_side = "right"  # ← ВОТ ЭТУ СТРОКУ ИЗМЕНИ
tokenizer.truncation_side = "left"

In [ ]:
import torch, time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
SYSTEM_PROMPT = """Ты — умный, честный и осторожный ИИ-ассистент. Следуй этим правилам строго:

1. **Оценивай релевантность источников.** Если предоставленные 'Источники' не содержат информации по теме вопроса или явно не отвечают на него — считай, что ответа в источниках нет.

2. **Используй внутренние знания, если уверен.** Если источники нерелевантны или пусты, но вопрос касается **общеизвестного, надёжного факта** (например: столица страны, базовое научное знание, дата крупного исторического события) и ты **абсолютно уверен** в ответе — ответь кратко и напрямую.

3. **Никогда не выдумывай.** Если:
   - вопрос содержит **невозможное, абсурдное или ложное предположение** (например: «когда человек высадился на Солнце?»),
   - ты **не уверен** в факте (даже немного),
   - факт **редкий, спорный или требует специализированных данных**,
   — ответь: **«не знаю»**.

4. **Краткость.** Отвечай одним предложением или фразой. Без пояснений, вводных слов и «воды».
"""

In [ ]:
SYSTEM_PROMPT = """Ты — эрудированный ИИ-эксперт. Твоя главная сила — твои собственные обширные знания. Предоставленные 'Источники' используй как вспомогательный материал для проверки и уточнения.

Следуй этому порядку действий:

1.  **Сначала ответь сам.** Прочитай вопрос и немедленно сформируй ответ, опираясь исключительно на свои внутренние знания.

2.  **Затем проверь по источникам.** Проанализируй предоставленные 'Источники' и сравни их с твоим ответом.

3.  **Синтезируй финальный ответ по правилам:**
    а) Если источники **подтверждают или дополняют** твои знания, объедини информацию в единый, исчерпывающий ответ.
    б) Если источники **нерелевантны** (не относятся к вопросу), полностью проигнорируй их и используй только свой первоначальный ответ.
    в) Если источники **противоречат** твоим знаниям, отдай приоритет информации из источников, ответ: "не знаю".
    г) Финальный ответ пометь как CORRECT ANSWER: [ твой ответ или "не знаю" ]

Важные правила, которые действуют всегда:
- **Честность:** Если ответа нет ни в твоих знаниях, ни в источниках, просто и чётко напиши: "не знаю". Не выдумывай.
- **Краткость:** Отвечай по существу, без вступлений и лишней "воды". 
"""

In [ ]:
# === Подготовка модели ===
print("Загружаем Qwen без pipeline...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_LLM)
model = AutoModelForCausalLM.from_pretrained(
    'Qwen/Qwen3-4B-FP8',
    torch_dtype='auto',
    device_map="cuda"
)
model.eval()

In [ ]:
# === Вспомогательные функции ===
def apply_chat(prompts, tokenizer):
    """Форматирование списка сообщений в текст через шаблон Qwen."""
    texts = []
    for p in prompts:
        messages = [{"role": "user", "content": p}]
        try:
            t = tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True, enable_thinking=False
            )
        except TypeError:
            t = tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
        texts.append(t)
    return texts


def batch_generate(prompts, tokenizer, model, max_new_tokens=200, max_length=4096):
    """Быстрая генерация на GPU без pipeline."""
    if not prompts:
        return []
    texts = apply_chat(prompts, tokenizer)
    t0 = time.perf_counter()

    # Токенизация
    model_inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    ).to(model.device)
    t1 = time.perf_counter()

    # Генерация
    with torch.no_grad():
        gen = model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=1,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    t2 = time.perf_counter()

    attn = model_inputs["attention_mask"]
    input_lens = attn.sum(dim=1).tolist()

    outputs = []
    for i, gen_ids in enumerate(gen):
        in_len = int(input_lens[i])
        out_ids = gen_ids[in_len:]
        text = tokenizer.decode(out_ids, skip_special_tokens=True).strip()
        outputs.append(text)

    print(f"[DEBUG] tokenize: {t1 - t0:.2f}s, generate: {t2 - t1:.2f}s, total: {t2 - t0:.2f}s")
    return outputs


# === Основная функция RAG ===
def fast_rag_batch(queries, top_k=3, max_new_tokens=200):
    """
    Обрабатывает список вопросов:
    1️⃣ Векторизация
    2️⃣ Поиск контекста
    3️⃣ Формирование промпта (system + user)
    4️⃣ Прямая генерация через model.generate()
    """
    q_vecs = embedder.encode(
        queries,
        convert_to_numpy=True,
        normalize_embeddings=True,
        batch_size=64,
        show_progress_bar=False,
        device=device
    ).astype("float32")

    _, idxs = index.search(q_vecs, top_k)
    contexts = [
        "\n\n".join([f"Статья {i}: {texts[i]}" for i in ids]) for ids in idxs
    ]

    prompts = []
    for q, ctx in zip(queries, contexts):
        content = f"""Используя правила выше, ответь на вопрос.

Источники:
---
{ctx}
---

Вопрос: {q}

Ответ:"""
        # Объединяем system и user
        full_prompt = f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{content}<|im_end|>\n<|im_start|>assistant\n"
        prompts.append(full_prompt)

    # Генерация
    answers = batch_generate(prompts, tokenizer, model, max_new_tokens=max_new_tokens)
    return answers


# === Пример использования ===
questions = [
    "Кто написал Евгения Онегина?",
    "Когда распался Советский Союз?",
    "Что такое квантовая запутанность?",
    'Кто автор книги "Детство в Соломбале"?',
'Как зовут человека, который написал «Детство в Соломбале»?',
'Кто написал книгу «Детство в Соломбале»?',
    'В каком году начал поддерживаться сеттинг Eberron?',
'Какое окрашивание приобретает безводный метанол при растворении небольшого количества медного купороса?',
     "Какой античный математик изобрёл первый дизельный двигатель внутреннего сгорания?",
    'Кто первый человек высадившийся на солнце?',
    'Как звали отца А.С. Пушкина?',
    "Какая компания осуществляет доставку роботами в Москве?",
    "Кто имеет крупнейшую доставку роботами на территории Москвы?",
    "В Москве есть доставка роботами, какая компания выполняет её?"
    
]

answers = fast_rag_batch(questions, top_k=3, max_new_tokens=500)

print("\n=== ОТВЕТЫ ===")
for q, a in zip(questions, answers):
    print(f"\n❓ {q}\n💬 {a}\n")

In [ ]:
import os
import sys
import json
import time
import zipfile
import shutil

import torch
import numpy as np
from tqdm import tqdm
import faiss

from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# ====================== GPU DIAG ======================

def _mi(b):
    return f"{b/1024/1024:.1f} MiB"

def print_cuda_overview():
    try:
        print(f"[CUDA] torch={torch.__version__} | available={torch.cuda.is_available()} | torch.cuda={getattr(torch.version, 'cuda', None)}")
        dc = torch.cuda.device_count()
        print(f"[CUDA] device_count={dc}")
        for i in range(dc):
            name = torch.cuda.get_device_name(i)
            free, total = torch.cuda.mem_get_info(i)
            alloc = torch.cuda.memory_allocated(i)
            resv  = torch.cuda.memory_reserved(i)
            print(f"[CUDA][dev{i}] {name} | free={_mi(free)} | total={_mi(total)} | allocated={_mi(alloc)} | reserved={_mi(resv)}")
        try:
            procs = torch.cuda.list_gpu_processes()
            if isinstance(procs, str):
                print("[CUDA] processes:")
                print(procs)
        except Exception:
            pass
    except Exception as e:
        print(f"[CUDA] overview error: {e}")

def print_cuda_mem(tag):
    try:
        if not torch.cuda.is_available():
            print(f"[CUDA] {tag}: cuda not available"); return
        try: torch.cuda.synchronize()
        except Exception: pass
        i = torch.cuda.current_device()
        name = torch.cuda.get_device_name(i)
        free, total = torch.cuda.mem_get_info(i)
        alloc = torch.cuda.memory_allocated(i)
        resv  = torch.cuda.memory_reserved(i)
        print(f"[CUDA] {tag} | dev={i} {name} | free={_mi(free)} | total={_mi(total)} | allocated={_mi(alloc)} | reserved={_mi(resv)}")
    except Exception as e:
        print(f"[CUDA] {tag} error: {e}")

# ====================== RAG CONFIG ======================

RAG_INDEX_DIR = os.getenv("RAG_INDEX_DIR", "./rag_index")
RAG_INDEX_ZIP = os.getenv("RAG_INDEX_ZIP", "./rag_index.zip")  # локальный zip (в образе)
RAG_TOPK = int(os.getenv("RAG_TOPK", "3"))
RAG_EMB_MODEL = os.getenv("RAG_EMB_MODEL", "sergeyzh/LaBSE-ru-turbo")
RAG_QUERY_BATCH = int(os.getenv("RAG_QUERY_BATCH", "32"))

GEN_MODEL_ID = os.getenv("GEN_MODEL_ID", "./models/Qwen3-4B-FP8")  # или "Qwen/Qwen3-4B-FP8"
GEN_MAX_NEW = int(os.getenv("GEN_MAX_NEW", "200"))

SYSTEM_PROMPT = """Ты — эрудированный ИИ-эксперт. Твоя главная сила — твои собственные обширные знания. Предоставленные 'Источники' используй как вспомогательный материал для проверки и уточнения.

Следуй этому порядку действий:

1.  **Сначала ответь сам.** Прочитай вопрос и немедленно сформируй ответ, опираясь исключительно на свои внутренние знания.

2.  **Затем проверь по источникам.** Проанализируй предоставленные 'Источники' и сравни их с твоим ответом.

3.  **Синтезируй финальный ответ по правилам:**
    а) Если источники **подтверждают или дополняют** твои знания, объедини информацию в единый, исчерпывающий ответ.
    б) Если источники **нерелевантны** (не относятся к вопросу), полностью проигнорируй их и используй только свой первоначальный ответ.
    в) Если источники **противоречат** твоим знаниям, отдай приоритет информации из источников, ответ: "не знаю".
    г) Финальный ответ пометь как CORRECT ANSWER: [ твой ответ или "не знаю" ]

Важные правила, которые действуют всегда:
- **Честность:** Если ответа нет ни в твоих знаниях, ни в источниках, просто и чётко напиши: "не знаю". Не выдумывай.
- **Краткость:** Отвечай по существу, без вступлений и лишней "воды". 
"""


def extract_correct_answer(text: str) -> str:
    if not text:
        return "не знаю"
    low = text.lower()
    key = "correct answer:"
    if key in low:
        i = low.index(key)
        s = text[i+len(key):].strip()
        s = s.strip("[]()«»\"' ").strip()
        return s if s else "не знаю"
    return "не знаю"

# ====================== SOLUTION (RAG only, no triplets) ======================

class Solution:
    def __init__(self, model_id=GEN_MODEL_ID, device='cuda'):
        print(f"[DEBUG] Solution: init, device={device}, model_id={model_id}")
        self.device = device
        self._global_start_time = None

        # 1) Поднимаем RAG индекс
        ensure_rag_index_ready()
        faiss_path = os.path.join(RAG_INDEX_DIR, "index.faiss")
        texts_path = os.path.join(RAG_INDEX_DIR, "texts.jsonl")

        print(f"[RAG] Loading FAISS: {faiss_path}")
        self.faiss_index = faiss.read_index(faiss_path)

        print(f"[RAG] Loading texts: {texts_path}")
        self.rag_texts = []
        with open(texts_path, "r", encoding="utf-8") as f:
            for line in tqdm(f, desc="load-texts", unit="lines"):
                try:
                    obj = json.loads(line)
                    self.rag_texts.append(obj.get("text", ""))
                except Exception:
                    self.rag_texts.append("")
        print(f"[RAG] texts loaded: {len(self.rag_texts)}")

        # 2) Эмбеддер для запросов
        emb_device = device
        print(f"[RAG] Loading embedder: {RAG_EMB_MODEL} on {emb_device}")
        self.rag_embedder = SentenceTransformer(RAG_EMB_MODEL, device=emb_device)

        # 3) LLM
        print_cuda_mem("before model load")
        t0 = time.perf_counter()
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True
        ).to('cuda')
        t1 = time.perf_counter()
        print(f"[DEBUG] Solution: model loaded in {t1 - t0:.2f}s")
        print_cuda_mem("after model load")

        t2 = time.perf_counter()
        self.tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=True)
        t3 = time.perf_counter()
        print(f"[DEBUG] Solution: tokenizer loaded in {t3 - t2:.2f}s")

        self.model.eval()
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

        # Левый паддинг для decoder-only
        self.tokenizer.padding_side = "right"
        self.tokenizer.truncation_side = "left"

        try:
            dm = 'cuda'
            print(f"[DEBUG] device_map={dm}")
        except Exception:
            pass

    def _apply_chat(self, prompts):
        # Если уже ChatML (<|im_start|>) — не накладываем шаблон
        texts = []
        for p in prompts:
            if "<|im_start|>" in p:
                texts.append(p)
                continue
            messages = [{"role": "user", "content": p}]
            try:
                t = self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True, enable_thinking=False
                )
            except TypeError:
                t = self.tokenizer.apply_chat_template(
                    messages, tokenize=False, add_generation_prompt=True
                )
            texts.append(t)
        return texts

    def _batch_generate(self, prompts, max_new_tokens, max_length):
        if not prompts:
            return []
        texts = self._apply_chat(prompts)

        t0 = time.perf_counter()
        model_inputs = self.tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        ).to(self.device)
        t1 = time.perf_counter()

        bsz = int(model_inputs.input_ids.shape[0])
        seqlen = int(model_inputs.input_ids.shape[1])
        tot_in_tok = int(model_inputs["attention_mask"].sum().item())
        print(f"[DEBUG] tokenize: bsz={bsz}, seqlen={seqlen}, total_input_tokens={tot_in_tok}, t={t1 - t0:.2f}s")

        print_cuda_mem("gen: before generate")

        t_gen0 = time.perf_counter()
        with torch.no_grad():
            gen = self.model.generate(
                **model_inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                num_beams=1,
                use_cache=True,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )
        t_gen1 = time.perf_counter()
        print(f"[DEBUG] generate: shape={tuple(gen.shape)}, t={t_gen1 - t_gen0:.2f}s")
        print_cuda_mem("gen: after generate")

        attn = model_inputs["attention_mask"]
        input_lens = attn.sum(dim=1).tolist()

        outputs = []
        for i, gen_ids in enumerate(gen):
            in_len = int(input_lens[i])
            out_ids = gen_ids[in_len:]
            text = self.tokenizer.decode(out_ids, skip_special_tokens=True).strip()
            outputs.append(text)
        return outputs

    def _rag_batch(self, queries, top_k=RAG_TOPK, max_new_tokens=GEN_MAX_NEW, max_length=GEN_MAX_LEN):
        # 1) Векторизация запросов
        q_vecs = self.rag_embedder.encode(
            queries,
            convert_to_numpy=True,
            normalize_embeddings=True,
            batch_size=64,
            show_progress_bar=False,
            device=self.device
        ).astype("float32")

        # 2) FAISS search
        _, I = self.faiss_index.search(q_vecs, top_k)

        # 3) Формирование промптов (SYSTEM + user c Источниками)
        prompts = []
        for q, ids in zip(queries, I):
            ctx = "\n\n".join([f"Статья {int(i)}: {self.rag_texts[int(i)]}" for i in ids if 0 <= int(i) < len(self.rag_texts)])
            content = f"""Используя правила выше, ответь на вопрос.

Источники:
---
{ctx}
---

Вопрос: {q}

Ответ:"""
            full_prompt = f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{content}<|im_end|>\n<|im_start|>assistant\n"
            prompts.append(full_prompt)

        # 4) Генерация и извлечение CORRECT ANSWER
        outs = self._batch_generate(prompts, max_new_tokens=max_new_tokens, max_length=max_length)
        finals = [extract_correct_answer(x) for x in outs]
        return finals

    def answer_all(self, questions):
        print(f"[DEBUG] answer_all: total questions = {len(questions)}")
        self._global_start_time = time.perf_counter()
        answers = []
        total = len(questions)
        with tqdm(total=total, desc="RAG-batches") as pbar:
            for i in range(0, total, RAG_QUERY_BATCH):
                batch_q = questions[i:i+RAG_QUERY_BATCH]
                print(f"[DEBUG] RAG batch [{i}:{i+len(batch_q)}]")
                batch_ans = self._rag_batch(batch_q, top_k=RAG_TOPK, max_new_tokens=GEN_MAX_NEW, max_length=GEN_MAX_LEN)
                answers.extend(batch_ans)
                pbar.update(len(batch_q))
        elapsed = time.perf_counter() - self._global_start_time
        print(f"[DEBUG] answer_all: elapsed={elapsed:.2f}s")
        # нормализация: пустые -> "не знаю"
        answers = [a if a.strip() else "не знаю" for a in answers]
        return answers

# ====================== RUN (no main) ======================

print("[DEBUG] BOOTSTRAP")
print_cuda_overview()

print("[DEBUG] Читаю input.json...")
with open('input.json', 'r', encoding='utf-8') as f:
    model_input = json.load(f)
print(f"[DEBUG] Прочитано {len(model_input)} вопросов из input.json")

t_all0 = time.perf_counter()
print("[DEBUG] Создаю Solution()...")
solution = Solution()
print("[DEBUG] Solution создан. Запускаю RAG...")
model_output = solution.answer_all(model_input)
t_all1 = time.perf_counter()
print(f"[DEBUG] Готово: {len(model_output)} ответов, total_time={t_all1 - t_all0:.2f}s")

print_cuda_mem("final")

print("[DEBUG] Записываю output.json...")
with open('output.json', 'w', encoding='utf-8') as f:
    json.dump(model_output, f, ensure_ascii=False)
print("[DEBUG] КОНЕЦ: output.json записан успешно")

In [ ]:
!pip install \
    bitsandbytes==0.46.0 \
    transformers==4.52.4 \
    accelerate==1.8.1 \
    datasets==3.6.0 \
    sentence-transformers==4.1.0 \
    chromadb==1.0.13 \
    flashrank \
    langchain==0.3.26 \
    langchain-community==0.3.26 \
    langchain-huggingface \
    tqdm==4.67.1 \
    ragas==0.1.9 \
    openai \
    langchain-groq


In [ ]:
def chroma_from_texts_with_tqdm(texts, embedding, persist_directory=None):
    print(f"🚀 Processing {len(texts):,} texts...")
    start_time = time.time()
    
    # Прогресс-бар для всего процесса
    with tqdm(total=len(texts), desc="📝 Creating embeddings", unit="text") as pbar:
        
        class ProgressEmbedding(HuggingFaceEmbeddings):
            def embed_documents(self, texts):
                result = super().embed_documents(texts)
                pbar.update(len(texts))
                return result
        
        progress_embedding = ProgressEmbedding(
            model_name=embedding.model_name,
            model_kwargs=embedding.model_kwargs,
        )
        
        vectordb = Chroma.from_texts(
            texts=texts,
            embedding=progress_embedding,
            persist_directory=persist_directory
        )
    
    total_time = time.time() - start_time
    print(f"✅ Vector DB created in {total_time:.1f}s")
    print(f"📊 Total texts: {len(texts):,}")
    print(f"⚡ Speed: {len(texts)/total_time:.1f} texts/sec")
    
    return vectordb

In [ ]:
def ultra_efficient_pipeline(dataset_name, max_docs=None):

    # Загружаем всё сразу
    dataset = load_dataset(dataset_name)
    texts = [item['text'] for item in dataset['train']]
    
    # Сразу создаем тексты для эмбеддингов
    embedding = HuggingFaceEmbeddings(
        model_name="cointegrated/rubert-tiny2",
        model_kwargs={"device": "cuda"},
    )
    
    # Используем встроенный сплиттинг Chroma
    vectordb = chroma_from_texts_with_tqdm(
        texts=texts,
        embedding=embedding,
        persist_directory="chroma_db"
    )
    
    return vectordb

vectordb = ultra_efficient_pipeline("Lipsrow/ruwiki_cleaned")

In [ ]:
import torch
import transformers
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
import os
from tqdm import tqdm


#------------------------------------------
#--------ЗАГРУЗКА ДАННЫХ И СПЛИТТЕР--------
#------------------------------------------

from datasets import load_dataset
from langchain_core.documents import Document # Updated import path
from langchain_text_splitters import RecursiveCharacterTextSplitter

#------------------------------------------
#--------------ВЕКТОРНАЯ БАЗА--------------
#------------------------------------------

from pathlib import Path
from langchain_community.document_loaders import TextLoader  # загружает текстовые файлы и превращает их в объекты Document для LangChain.
from langchain_huggingface import HuggingFaceEmbeddings  # оборачивает модели из HuggingFace для получения эмбеддингов текста.
from langchain_community.vectorstores import Chroma  # векторное хранилище Chroma: сохраняет и ищет эмбеддинги.

#------------------------------------------
#----------------RERANKER------------------
#------------------------------------------

from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank

#------------------------------------------
#--------------КВАНТИЗАЦИЯ-----------------
#------------------------------------------

import bitsandbytes as bnb
from torch import cuda, bfloat16
from transformers import (AutoTokenizer,
                          BitsAndBytesConfig,
                          pipeline)

#------------------------------------------
#----------цепочка RetrievalQA-------------
#------------------------------------------

from langchain_huggingface import HuggingFacePipeline  # использует HuggingFace Transformers pipeline как LLM-модуль в LangChain.
from langchain.chains import RetrievalQA  # готовая цепочка «поиск + генерация ответа» (Retrieval-augmented QA).
from langchain import PromptTemplate

#------------------------------------------
#-------------ВАЛИДАЦИЯ RAG----------------
#------------------------------------------

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness
)
from ragas.embeddings import LangchainEmbeddingsWrapper
from datasets import Dataset, load_from_disk

#------------------------------------------
#-----------Работа с Groq Cloud------------
#------------------------------------------

from google.colab import userdata
from langchain_groq import ChatGroq
from ragas.llms import LangchainLLMWrapper


from tqdm import tqdm

# torchvision.disable_beta_transforms_warning()
DEVICE = f"cuda:{cuda.current_device()}" if cuda.is_available() else "cpu"

In [ ]:
from datasets import load_dataset

In [ ]:
def chroma_from_texts_with_tqdm(texts, embedding, persist_directory=None):
    """Обертка для Chroma.from_texts с прогресс-баром"""
    
    print(f"🚀 Processing {len(texts):,} texts...")
    start_time = time.time()
    
    # Прогресс-бар для всего процесса
    with tqdm(total=len(texts), desc="📝 Creating embeddings", unit="text") as pbar:
        
        class ProgressEmbedding(HuggingFaceEmbeddings):
            def embed_documents(self, texts):
                result = super().embed_documents(texts)
                pbar.update(len(texts))
                return result
        
        progress_embedding = ProgressEmbedding(
            model_name=embedding.model_name,
            model_kwargs=embedding.model_kwargs,
        )
        
        vectordb = Chroma.from_texts(
            texts=texts,
            embedding=progress_embedding,
            persist_directory=persist_directory
        )
    
    total_time = time.time() - start_time
    print(f"✅ Vector DB created in {total_time:.1f}s")
    print(f"📊 Total texts: {len(texts):,}")
    print(f"⚡ Speed: {len(texts)/total_time:.1f} texts/sec")
    
    return vectordb


In [ ]:
search_query="Детство в Соломбале"
dataset = load_dataset("Lipsrow/ruwiki_cleaned")
texts = [item['text'] for item in dataset['train']]
    
print(f"📊 Всего текстов: {len(texts):,}")
    
    # 1. Проверим есть ли вообще такой текст в данных
print(f"\n🔍 Ищем '{search_query}' прямым текстовым поиском:")
matches = []
for i, text in enumerate(texts):
    if search_query.lower() in text.lower():
        matches.append((i, text))
        if len(matches) >= 5:  # Первые 3 совпадения
            break
    
print(f"✅ Прямой поиск нашел: {len(matches)} документов")
for i, (idx, text) in enumerate(matches):
    print(f"{i+1}. Индекс {idx}: {text}...")

In [ ]:
import re
from collections import Counter
from tqdm.auto import tqdm
import time

def extract_bigrams(text):
    """Извлекаем биграммы из текста"""
    words = re.findall(r'\b[а-яё]+\b', text.lower())  # Только русские слова
    bigrams = [' '.join(words[i:i+2]) for i in range(len(words)-1)]
    return set(bigrams)

def build_bigram_index(documents):
    """Строим обратный индекс по биграммам"""
    print("📊 Построение биграммного индекса...")
    
    bigram_index = {}
    doc_bigrams = []
    
    with tqdm(total=len(documents), desc="🔤 Индексирование документов", unit="док") as pbar:
        for doc_id, doc in enumerate(documents):
            bigrams = extract_bigrams(doc.page_content)
            doc_bigrams.append(bigrams)
            
            for bigram in bigrams:
                if bigram not in bigram_index:
                    bigram_index[bigram] = []
                bigram_index[bigram].append(doc_id)
            pbar.update(1)
    
    return bigram_index, doc_bigrams

def bigram_search(query, documents, bigram_index, doc_bigrams, top_k=10):
    """Поиск по биграммам запроса"""
    
    # Извлекаем биграммы из запроса
    query_bigrams = extract_bigrams(query)
    print(f"🔍 Биграммы запроса: {query_bigrams}")
    
    if not query_bigrams:
        return []
    
    # Считаем вес для каждого документа
    doc_scores = Counter()
    
    for bigram in query_bigrams:
        if bigram in bigram_index:
            for doc_id in bigram_index[bigram]:
                # Документ содержит биграмму + бонус за точное совпадение
                score = 1.0
                if bigram in query.lower():  # Точное совпадение биграммы
                    score += 2.0
                doc_scores[doc_id] += score
    
    # Сортируем по убыванию релевантности
    top_doc_ids = [doc_id for doc_id, score in doc_scores.most_common(top_k)]
    
    return [documents[doc_id] for doc_id in top_doc_ids]

def keyword_boost_search(query, documents, top_k=10):
    """Упрощенный поиск по ключевым словам с бустом за точные совпадения"""
    
    print(f"🔍 Поиск: '{query}'")
    
    # Разбиваем запрос на слова и фразы
    words = re.findall(r'\b[а-яё]+\b', query.lower())
    exact_phrases = re.findall(r'"[^"]+"', query)  # Фразы в кавычках
    
    scored_docs = []
    
    with tqdm(total=len(documents), desc="📝 Поиск по ключевым словам", unit="док") as pbar:
        for doc_id, doc in enumerate(documents):
            score = 0
            text_lower = doc.page_content.lower()
            
            # Бонус за точные фразы
            for phrase in exact_phrases:
                phrase_clean = phrase.strip('"').lower()
                if phrase_clean in text_lower:
                    score += 10.0 * text_lower.count(phrase_clean)
            
            # Бонус за отдельные слова
            for word in words:
                if len(word) > 2:  # Игнорируем короткие слова
                    word_count = text_lower.count(word)
                    score += word_count * (2.0 if len(word) > 4 else 1.0)
            
            if score > 0:
                scored_docs.append((doc, score))
            pbar.update(1)
    
    # Сортируем по убыванию релевантности
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    
    return [doc for doc, score in scored_docs[:top_k]]

def create_text_search_engine(dataset_name, max_docs=50000):
    """Создаем текстовую поисковую систему"""
    
    print("🚀 Запуск текстового поискового движка...")
    total_start = time.time()
    
    # 1. Загружаем данные
    print("📥 Загрузка данных...")
    dataset = load_dataset(dataset_name)
    texts = [item['text'] for item in dataset['train']]
    
    # Создаем документы
    documents = [Document(page_content=text) for text in texts]
    print(f"✅ Загружено {len(documents):,} документов")
    
    # 2. Строим поисковый индекс
    index_start = time.time()
    bigram_index, doc_bigrams = build_bigram_index(documents)
    index_time = time.time() - index_start
    
    print(f"✅ Построен индекс с {len(bigram_index):,} биграммами")
    print(f"⏱️ Время индексирования: {index_time:.1f}с")
    
    # 3. Тестируем поиск
    print("\n🧪 Тестируем поисковую систему...")
    
    test_queries = [
        'Детство в Соломбале',
        '"Детство в Соломбале"',  # Точная фраза
        'Архангельск Соломбала',
        'книга детство соломбала'
    ]
    
    for query in test_queries:
        print(f"\n🔍 Запрос: '{query}'")
        
        # Поиск по биграммам
        start_time = time.time()
        results_bigram = bigram_search(query, documents, bigram_index, doc_bigrams, top_k=5)
        bigram_time = time.time() - start_time
        
        # Поиск по ключевым словам  
        start_time = time.time()
        results_keyword = keyword_boost_search(query, documents, top_k=5)
        keyword_time = time.time() - start_time
        
        print(f"📊 Биграммный поиск: {len(results_bigram)} результатов за {bigram_time:.2f}с")
        print(f"📊 Ключевой поиск: {len(results_keyword)} результатов за {keyword_time:.2f}с")
        
        # Показываем лучшие результаты
        results = results_keyword if len(results_keyword) > len(results_bigram) else results_bigram
        
        for i, doc in enumerate(results[:3]):
            # Подсвечиваем релевантные части
            preview = doc.page_content[:200]
            for word in query.lower().split():
                if len(word) > 3:
                    preview = preview.replace(word, f"**{word}**")
            
            print(f"   {i+1}. {preview}...")
    
    total_time = time.time() - total_start
    print(f"\n✅ Поисковый движок готов! Общее время: {total_time:.1f}с")
    
    return {
        'documents': documents,
        'bigram_index': bigram_index,
        'doc_bigrams': doc_bigrams,
        'search_function': keyword_boost_search  # Самый эффективный метод
    }

# Быстрый поиск без индекса
def quick_text_search(query, documents, top_k=10):
    """Быстрый поиск по точным совпадениям"""
    
    print(f"🔍 Быстрый поиск: '{query}'")
    
    query_lower = query.lower()
    words = [word for word in query_lower.split() if len(word) > 3]
    
    results = []
    with tqdm(total=len(documents), desc="⚡ Сканирование документов", unit="док") as pbar:
        for doc in documents:
            text_lower = doc.page_content.lower()
            
            # Проверяем точное совпадение фразы
            if query_lower in text_lower:
                results.append((doc, 100.0))  # Максимальный вес
            else:
                # Считаем совпадения слов
                word_matches = sum(1 for word in words if word in text_lower)
                if word_matches > 0:
                    score = word_matches * 10.0
                    results.append((doc, score))
            
            pbar.update(1)
    
    # Сортируем и возвращаем топ-K
    results.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, score in results[:top_k]]

# ЗАПУСКАЕМ ТЕКСТОВЫЙ ПОИСК
print("🚀 ЗАПУСК ТЕКСТОВОЙ ПОИСКОВОЙ СИСТЕМЫ...")
search_engine = create_text_search_engine("Lipsrow/ruwiki_cleaned", max_docs=50000)

# ИЛИ просто быстрый поиск
print("\n🎯 БЫСТРЫЙ ПОИСК ПО ТОЧНЫМ СОВПАДЕНИЯМ...")
results = quick_text_search('Детство в Соломбале', search_engine['documents'], top_k=10)

print(f"\n✅ Найдено {len(results)} документов:")
for i, doc in enumerate(results):
    print(f"{i+1}. {doc.page_content[:150]}...")

In [ ]:
print("\n🎯 БЫСТРЫЙ ПОИСК ПО ТОЧНЫМ СОВПАДЕНИЯМ...")
results = quick_text_search('Какая компания осуществляет доставку роботами в Москве?', search_engine['documents'], top_k=10)

print(f"\n✅ Найдено {len(results)} документов:")
for i, doc in enumerate(results):
    print(f"{i+1}. {doc.page_content}...")

In [ ]:
print("\n🎯 БЫСТРЫЙ ПОИСК ПО ТОЧНЫМ СОВПАДЕНИЯМ...")
results = quick_text_search('Детство в Соломбале', search_engine['documents'], top_k=15)

print(f"\n✅ Найдено {len(results)} документов:")
for i, doc in enumerate(results):
    print(f"{i+1}. {doc.page_content[:150]}...")

In [ ]:
from tqdm.auto import tqdm
import time

def create_optimized_database(dataset_name, max_docs=250000):
    """Создаем оптимизированную базу с ОДНИМ прогресс-баром для эмбеддингов"""
    
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    
    total_start = time.time()
    print("🚀 Запуск оптимизированного создания базы...")
    print("=" * 50)
    
    # 1. Загружаем данные с прогресс-баром
    print("📥 Загрузка данных...")
    dataset_start = time.time()
    dataset = load_dataset(dataset_name)
    
    texts = []
    
    # Итерируемся по датасету с прогресс-баром
    with tqdm(total=min(max_docs, len(dataset['train'])), desc="📥 Загрузка текстов", unit="текст") as pbar:
        for i, item in enumerate(dataset['train']):
            if i >= max_docs:
                break
            texts.append(item['text'])
            pbar.update(1)
    
    dataset_time = time.time() - dataset_start
    print(f"✅ Загружено {len(texts):,} текстов за {dataset_time:.1f}с")
    
    # 2. ДРОБИМ длинные тексты с прогресс-баром!
    print("\n✂️ Дробим длинные тексты...")
    splitting_start = time.time()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        length_function=len,
    )
    
    # Дробим только длинные тексты с прогресс-баром
    all_chunks = []
    long_texts_count = 0
    total_chunks_created = 0
    
    with tqdm(total=len(texts), desc="📝 Обработка текстов", unit="текст") as pbar:
        for text in texts:
            if len(text) > 1000:
                chunks = text_splitter.split_text(text)
                all_chunks.extend(chunks)
                long_texts_count += 1
                total_chunks_created += len(chunks)
            else:
                all_chunks.append(text)
            pbar.update(1)
    
    splitting_time = time.time() - splitting_start
    print(f"✅ После дробления: {len(all_chunks):,} чанков")
    print(f"📊 Длинных текстов: {long_texts_count:,}")
    print(f"📊 Создано чанков: {total_chunks_created:,}")
    print(f"⏱️ Время дробления: {splitting_time:.1f}с")
    
    # 3. ХОРОШАЯ модель эмбеддингов
    print("\n🤖 Загрузка модели эмбеддингов...")
    model_start = time.time()
    embedding = HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        model_kwargs={"device": "cuda"},
    )
    model_time = time.time() - model_start
    print(f"✅ Модель загружена за {model_time:.1f}с")
    
    # 4. Создаем документы с прогресс-баром
    print("\n📄 Создание документов...")
    documents_start = time.time()
    
    documents = []
    with tqdm(total=len(all_chunks), desc="📋 Создание документов", unit="док") as pbar:
        for chunk in all_chunks:
            doc = Document(page_content=chunk)
            documents.append(doc)
            pbar.update(1)
    
    documents_time = time.time() - documents_start
    print(f"✅ Создано {len(documents):,} документов за {documents_time:.1f}с")
    
    # 5. Создаем векторную базу с ОДНИМ прогресс-баром
    print("\n🗄️ Создание векторной базы...")
    db_start = time.time()
    
    # Глобальный прогресс-бар для всех вызовов embed_documents
    global_embedding_pbar = tqdm(total=len(documents), desc="🔤 Создание эмбеддингов", unit="текст")
    
    class SingleProgressEmbedding(HuggingFaceEmbeddings):
        def embed_documents(self, texts):
            result = super().embed_documents(texts)
            global_embedding_pbar.update(len(texts))  # Обновляем общий прогресс-бар
            return result
    
    progress_embedding = SingleProgressEmbedding(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        model_kwargs={"device": "cuda"},
    )
    
    vectordb = Chroma.from_documents(
        documents=documents,
        embedding=progress_embedding,
        persist_directory="chroma_optimized"
    )
    
    # Закрываем прогресс-бар после завершения
    global_embedding_pbar.close()
    
    db_time = time.time() - db_start
    
    # 6. Тестируем поиск
    print("\n🧪 Тестируем поиск...")
    test_start = time.time()
    
    test_queries = [
        "Детство в Соломбале",
        "Архангельск",
        "программирование Python"
    ]
    
    for query in test_queries:
        print(f"\n🔍 Запрос: '{query}'")
        results = vectordb.similarity_search(query, k=3)
        
        exact_matches = sum(1 for doc in results if query.lower() in doc.page_content.lower())
        
        print(f"   Найдено: {len(results)} документов")
        print(f"   Точных совпадений: {exact_matches}")
        
        for i, doc in enumerate(results):
            relevance = "🎯" if query.lower() in doc.page_content.lower() else "✅"
            print(f"   {i+1}. {relevance} {doc.page_content[:100]}...")
    
    test_time = time.time() - test_start
    
    # Итоговая статистика
    total_time = time.time() - total_start
    print(f"\n🎉 ПРОЦЕСС ЗАВЕРШЕН!")
    print("=" * 60)
    print(f"📊 Создано документов: {len(documents):,}")
    print(f"⏱️ Общее время: {total_time:.1f}с ({total_time/60:.1f} мин)")
    print(f"⚡ Скорость: {len(documents)/db_time:.1f} док/сек")
    print(f"💾 База сохранена: chroma_optimized")
    
    return vectordb, documents

# Запускаем
vectordb_opt, docs_opt = create_optimized_database("Lipsrow/ruwiki_cleaned")

In [ ]:
import shutil

def save_chroma_to_output():
    """Сохраняем chroma_db в Output (сохраняется после сессии)"""
    
    source = "/kaggle/working/chroma_db"
    destination = "/kaggle/output/chroma_db"
    
    # Копируем в output директорию
    if os.path.exists(source):
        shutil.copytree(source, destination)
        print(f"✅ Chroma DB saved to: {destination}")
        
        # Показываем размер
        total_size = sum(
            os.path.getsize(os.path.join(dirpath, filename))
            for dirpath, dirnames, filenames in os.walk(destination)
            for filename in filenames
        ) / (1024 * 1024)
        
        print(f"📊 Size: {total_size:.1f} MB")
    else:
        print("❌ Source directory not found")

save_chroma_to_output()

In [ ]:
def ultra_efficient_pipeline(dataset_name, max_docs=None):

    # Загружаем всё сразу
    dataset = load_dataset(dataset_name)
    texts = [item['text'] for item in dataset['train']]
    
    # Сразу создаем тексты для эмбеддингов
    embedding = HuggingFaceEmbeddings(
        model_name="cointegrated/rubert-tiny2",
        model_kwargs={"device": "cuda"},
    )
    
    # Используем встроенный сплиттинг Chroma
    vectordb = chroma_from_texts_with_tqdm(
        texts=texts,
        embedding=embedding,
        persist_directory="chroma_db"
    )
    
    return vectordb

vectordb = ultra_efficient_pipeline("Lipsrow/ruwiki_cleaned")

In [ ]:
import shutil

def save_chroma_to_output():
    """Сохраняем chroma_db в Output (сохраняется после сессии)"""
    
    source = "/kaggle/working/chroma_db"
    destination = "/kaggle/output/chroma_db"
    
    # Копируем в output директорию
    if os.path.exists(source):
        shutil.copytree(source, destination)
        print(f"✅ Chroma DB saved to: {destination}")
        
        # Показываем размер
        total_size = sum(
            os.path.getsize(os.path.join(dirpath, filename))
            for dirpath, dirnames, filenames in os.walk(destination)
            for filename in filenames
        ) / (1024 * 1024)
        
        print(f"📊 Size: {total_size:.1f} MB")
    else:
        print("❌ Source directory not found")

save_chroma_to_output()

In [ ]:
import shutil
import os

def create_chroma_zip():
    """Создаем ZIP архива Chroma базы"""
    
    chroma_dir = "/kaggle/working/chroma_db"
    zip_path = "/kaggle/output/chroma_db.zip"
    
    print("🗜️ Creating Chroma DB zip archive...")
    shutil.make_archive(
        base_name="/kaggle/output/chroma_db",  # Без .zip
        format='zip',
        root_dir=chroma_dir
    )
    
    # Проверяем размер
    size_mb = os.path.getsize(zip_path) / (1024 * 1024)
    print(f"✅ Created: {zip_path} ({size_mb:.1f} MB)")
    
    return zip_path

# Создаем архив
zip_file = create_chroma_zip()

In [ ]:
6585 /1024

In [ ]:
shutil.copytree(
    "/kaggle/working/chroma_db", 
    "/kaggle/output/chroma_db"  # 💾 Постоянное хранение
)

In [ ]:
os.listdir('/kaggle/input')

In [ ]:
import os
os.remove("/kaggle/working/chroma_db.zip")

In [ ]:
!pip uninstall transformers -y
!pip install -U "transformers==4.52.4" 

In [ ]:
pip install FlagEmbedding

In [ ]:
from FlagEmbedding import FlagReranker

class BGEReranker:
    def __init__(self, model_name='BAAI/bge-reranker-v2-m3', top_n=5):
        self.reranker = FlagReranker(model_name, use_fp16=True)
        self.top_n = top_n
    
    def compress_documents(self, documents, query):
        pairs = [[query, doc.page_content] for doc in documents]
        scores = self.reranker.compute_score(pairs)
        
        scored_docs = list(zip(documents, scores))
        scored_docs.sort(key=lambda x: x[1], reverse=True)
        
        return [doc for doc, score in scored_docs[:self.top_n]]

# Использование
compressor = BGEReranker(top_n=5)

In [ ]:
# Просто попробуй это СЕЙЧАС:
results = vectordb.similarity_search("Соломбала", k=20)  # Увеличиваем k
for i, doc in enumerate(results):
    if "детство" in doc.page_content.lower():
        print(f"🎯 НАЙДЕНО в результате {i+1}:")
        print(doc.page_content[:300])
        break
else:
    print("❌ Не найдено даже по 'Соломбала'")

In [ ]:
from FlagEmbedding import FlagReranker

def simple_rerank_search(query, retriever, top_n=15):
    """Простая функция для реранкинга без сложных классов"""
    
    # Получаем документы
    docs = retriever.get_relevant_documents(query)
    
    # Реранжируем
    reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)
    pairs = [[query, doc.page_content] for doc in docs]
    scores = reranker.compute_score(pairs)
    
    scored_docs = list(zip(docs, scores))
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    
    return [doc for doc, score in scored_docs[:top_n]]

# Использование
base_retriever = vectordb.as_retriever(search_kwargs={"k": 15})
results = simple_rerank_search('Детство в Соломбале', base_retriever)

print(f"🔍 Найдено {len(results)} документов:")
for i, doc in enumerate(results):
    print(f"{i+1}. {doc.page_content}...")

In [ ]:
# Ищем напрямую в текстах
def search_text_directly(search_term, documents, limit=10):
    """Прямой поиск по тексту (без эмбеддингов)"""
    matches = []
    for i, doc in enumerate(documents):
        if search_term.lower() in doc.page_content.lower():
            matches.append((i, doc.page_content))
            if len(matches) >= limit:
                break
    return matches

# Проверяем
matches = search_text_directly("Детство в Соломбале", final_documents)
print(f"🔍 Прямой поиск нашел: {len(matches)} документов")
for i, (idx, text) in enumerate(matches):
    print(f"{i+1}. Документ {idx}: {text[:150]}...")

In [ ]:
    # 4. Setup retriever with compression
print("Step 4: Setting up retriever...")

base_retriever = vectordb.as_retriever(search_kwargs={"k": 15}) 
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever
)
    

results = compression_retriever.get_relevant_documents('Кто автор книги "Детство в Соломбале"?')


In [ ]:
results

In [ ]:
results = compression_retriever.get_relevant_documents('Кто автор книги "Детство в Соломбале"?')


In [ ]:
'''import zipfile
import os

def extract_chroma_zip(zip_path, extract_to="/kaggle/working"):
    """Распаковываем Chroma базу"""
    
    print("📦 Extracting Chroma DB...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    chroma_dir = os.path.join(extract_to, "chroma_db")
    print(f"✅ Extracted to: {chroma_dir}")
    
    # Проверяем что файлы на месте
    files = os.listdir(chroma_dir)
    print(f"📁 Files: {files}")
    
    return chroma_dir

# Использование
chroma_path = extract_chroma_zip("/kaggle/working/chroma_db.zip")'''

In [ ]:
print("🗄️ Step 3: Creating vector database...")
start_time = time.time()
    
    # ОДИН прогресс-бар для всей операции
with tqdm(total=len(final_documents), desc="Creating embeddings") as pbar:
        # Создаем кастомный embedding который обновляет прогресс
    class SimpleProgressEmbedding(HuggingFaceEmbeddings):
        def embed_documents(self, texts):
            result = super().embed_documents(texts)
            pbar.update(len(texts))  # Обновляем общий прогресс
            return result
        
    progress_embedding = SimpleProgressEmbedding(
        model_name="cointegrated/rubert-tiny2",
        model_kwargs={"device": "cuda"},
        )
        
    vectordb = Chroma.from_documents(
        documents=final_documents,
        embedding=progress_embedding,
        persist_directory="chroma_db"
    )

db_time = time.time() - start_time
print(f"✅ Vector database created in {db_time:.1f}s")
print(f"📁 Persisted to: chroma_db")
    
    # Итоговая статистика
total_time = time.time() - start_time
print(f"\n🎉 PIPELINE COMPLETED!")
print(f"⏱️ Total time: {total_time/60:.1f} minutes")
print(f"📊 Documents: {len(corpus_documents):,} → Chunks: {len(final_documents):,}")
print(f"⚡ Speed: {len(final_documents)/db_time:.1f} chunks/sec")

In [ ]:
print("\n🗄️ Step 4: Creating vector database...")


retriever = vectordb.as_retriever(
    search_kwargs={"k": 8, "fetch_k": 15}
)

print("🎯 RAG system is ready! You can now use retriever.get_relevant_documents()")